In [33]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [34]:
%load_ext autoreload
%autoreload 2 

import os
import sys

curr_dir = os.getcwd()
print('current direcotry is ', curr_dir)
if curr_dir not in sys.path:
    sys.path.append(curr_dir)
src_dir = os.path.join(curr_dir, 'src')
if src_dir not in sys.path:
    sys.path.append(src_dir)
print(sys.path)

#
pdb_dir = './pdb'
data_dir = './data'
outputs_dir = './outputs'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
current direcotry is  /home/yuan/bio/protein_design/docking_antigen-antibody/antibody_antigen_complex
['/home/yuan/anaconda3/envs/openfold-env/lib/python310.zip', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/lib-dynload', '', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages/openfold-2.2.0-py3.10-linux-x86_64.egg', '/home/yuan/bio/protein_design/docking_antigen-antibody/antibody_antigen_complex', '/home/yuan/bio/protein_design/docking_antigen-antibody/antibody_antigen_complex/src']


## view structure

In [35]:
pdb_acc = '7VKL'

In [3]:
# get *pdb
pdb_file = f'pdb/{pdb_acc}.pdb'
if not os.path.isfile(pdb_file):
    print(f"download {pdb_acc} to {pdb_file}")
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_acc}.pdb -P ./pdb")
print(f"{pdb_file} is ready")

download 7VKL to pdb/7VKL.pdb
pdb/7VKL.pdb is ready


In [4]:
import py3Dmol
# Create a view
view = py3Dmol.view(width=400, height=300)
view.addModel(open(f'pdb/{pdb_acc}.pdb').read(), 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## parse complex

In [36]:
# get chains
from src.parse_abag import ParseAbAg

p = ParseAbAg('/home/yuan/data/pdb/rsync/vk/pdb5vkh.ent.gz')
# p.structure.header

# get chains
chains = p.get_chains()
pprint(chains)

Successfully retrieve structure of 5VKH
[{'chains': [{'AA residues': 351,
              'chain_id': 'A',
              'seq': 'QVQLQQPGAELVKPGASVKLSCKASGYTFTSDWIHWVKQRPGHGLEWIGEIIPSYGRANYNEKIQKKATLTADKSSSTAFMQLSSLTSEDSAVYYCARERGDGYFAVWGAGTTVTVSSAKTTPPSVYPLAPGSAAQTNSMVTLGCLVKGYFPEPVTVTWNSGSLSSGVHTFPAVLQSDLYTLSSSVTVPSSSWPSETVTCNVAHPASSTKVDKKIVPRD'},
             {'AA residues': 336,
              'chain_id': 'B',
              'seq': 'DILLTQSPAILSVSPGERVSFSCRASQSIGTDIHWYQQRTNGSPRLLIKYASESISGIPSRFSGSGSGTDFTLSINSVESEDIANYYCQQSNRWPFTFGSGTKLEIKRADAAPTVSIFPPSSEQLTSGGASVVCFLNNFYPKDINVKWKIDGSERQNGVLNSWTDQDSKDSTYSMSSTLTLTKDEYERHNSYTCEATHKTSTSPIVKSFNRN'},
             {'AA residues': 158,
              'chain_id': 'C',
              'seq': 'SALHWRAAGAATVLLVIVLLAGSYLAVLAERGAPGAQLITYPRALWWSVETATTVGYGDLAPVTLWGRCVAVVVMVAGITSAGLVTAALATWFVGREQERRGH'}],
  'model_id': 0}]


In [23]:
p.structure.header['compound']

{'1': {'misc': '', 'molecule': 'antibody heavy chain', 'chain': 'a'},
 '2': {'misc': '', 'molecule': 'antibody light chain', 'chain': 'b'},
 '3': {'misc': '',
  'molecule': 'ph-gated potassium channel kcsa',
  'chain': 'c',
  'synonym': 'streptomyces lividans k+ channel,skc1',
  'engineered': 'yes'}}

In [37]:
# filter: is antibody-antigen complex
p.filter_antibody_antigen(outputs_dir)
p.update_chains('chain_type')
pprint(p.chains[0]['chains'][0])

outputs dir:  ./outputs/5V/5VKH
{'AA residues': 351,
 'chain_id': 'A',
 'chain_type': 'heavy',
 'seq': 'QVQLQQPGAELVKPGASVKLSCKASGYTFTSDWIHWVKQRPGHGLEWIGEIIPSYGRANYNEKIQKKATLTADKSSSTAFMQLSSLTSEDSAVYYCARERGDGYFAVWGAGTTVTVSSAKTTPPSVYPLAPGSAAQTNSMVTLGCLVKGYFPEPVTVTWNSGSLSSGVHTFPAVLQSDLYTLSSSVTVPSSSWPSETVTCNVAHPASSTKVDKKIVPRD'}


In [38]:
# split chains
p.chains_to_pdb()

Saved chain ['A'] to ./outputs/5V/5VKH/5VKH_A.pdb
Saved chain ['C'] to ./outputs/5V/5VKH/5VKH_C.pdb
Saved chain ['A', 'C'] to ./outputs/5V/5VKH/5VKH_A_C.pdb
Saved chain ['B'] to ./outputs/5V/5VKH/5VKH_B.pdb
Saved chain ['B', 'C'] to ./outputs/5V/5VKH/5VKH_B_C.pdb


In [26]:
p.bounded

[{'type': 'heavy-receptor',
  'chains': [['A'], ['C'], ['A', 'C']],
  'pdb_files': ['./outputs/5V/5VKH/5VKH_A.pdb',
   './outputs/5V/5VKH/5VKH_C.pdb',
   './outputs/5V/5VKH/5VKH_A_C.pdb']},
 {'type': 'light-receptor',
  'chains': [['B'], ['C'], ['B', 'C']],
  'pdb_files': ['./outputs/5V/5VKH/5VKH_B.pdb',
   './outputs/5V/5VKH/5VKH_C.pdb',
   './outputs/5V/5VKH/5VKH_B_C.pdb']}]

## load SASA

In [39]:
# calculate SASA
p.build_freesasa()

'./outputs/5V/5VKH/freesasa.p'

In [40]:
from analyze_sasa import AnalyzeSasa

p2= AnalyzeSasa('./outputs/5V/5VKH/freesasa.p')
p2.print_meta()

number of complex 2
heavy chain A ~ receptor chain C
light chain B ~ receptor chain C


In [30]:
# complex
p2.print_delta_sasa(0)

complex: heavy chain A ~ receptor chain C
------------------------------
heavy chain:
    chain_id res_no  res aa      value
29         A     30  THR  T  21.741028
30         A     31  SER  S  25.686868
32         A     33  TRP  W  56.461030
34         A     35  HIS  H   1.254828
49         A     50  GLU  E  24.326483
51         A     52  ILE  I  22.982987
53         A     54  SER  S   7.138050
54         A     55  TYR  Y  50.598868
56         A     57  ARG  R  48.903313
58         A     59  ASN  N  30.217170
61         A     62  GLU  E  41.781159
98         A     99  GLU  E  13.076917
99         A    100  ARG  R   1.998656
100        A    101  GLY  G  44.087453
101        A    102  ASP  D  30.175990
102        A    103  GLY  G  13.519404
------------------------------
receptor chain:
   chain_id res_no  res aa       value
23        C     45  TYR  Y   84.801513
24        C     46  LEU  L    4.055821
26        C     48  VAL  V    4.055821
27        C     49  LEU  L  107.627094
28       

In [31]:
# complex
p2.print_delta_sasa(1)

complex: light chain B ~ receptor chain C
------------------------------
light chain:
   chain_id res_no  res aa       value
0         B      1  ASP  D   16.341861
27        B     28  SER  S    1.998656
31        B     32  ASP  D   22.327827
49        B     50  TYR  Y   28.887472
90        B     91  SER  S   27.802693
91        B     92  ASN  N   46.295716
92        B     93  ARG  R   65.362821
93        B     94  TRP  W  144.855155
94        B     95  PRO  P    2.703881
95        B     96  PHE  F   48.938273
------------------------------
receptor chain:
   chain_id res_no  res aa      value
30        C     52  ARG  R  37.081676
31        C     53  GLY  G  39.564363
32        C     54  ALA  A   4.996640
33        C     55  PRO  P  41.910153
34        C     56  GLY  G  45.692429
35        C     57  ALA  A  14.343205
36        C     58  GLN  Q  67.038144
38        C     60  ILE  I  60.837319
39        C     61  THR  T  34.178266
41        C     63  PRO  P   1.351940
42        C     64  

In [32]:
# retrieve sequences
motifs = p2.retrieve_seq()
motifs

([{'heavy': [{'seq': 'TSDWIHWVKQRPGHGLEWIGEIIPSYGRANYNE',
     'sig_res': 11,
     'start': (29, '30'),
     'end': (61, '62')},
    {'seq': 'ERGDG', 'sig_res': 5, 'start': (98, '99'), 'end': (102, '103')}],
   'receptor': [{'seq': 'YLAVLAERGAPGAQLITYP',
     'sig_res': 11,
     'start': (23, '45'),
     'end': (41, '63')}]},
  {'light': [{'seq': 'DILLTQSPAILSVSPGERVSFSCRASQSIGTDIHWYQQRTNGSPRLLIKY',
     'sig_res': 4,
     'start': (0, '1'),
     'end': (49, '50')},
    {'seq': 'SNRWPF', 'sig_res': 6, 'start': (90, '91'), 'end': (95, '96')}],
   'receptor': [{'seq': 'RGAPGAQLITYPR',
     'sig_res': 11,
     'start': (30, '52'),
     'end': (42, '64')}]}],
 './outputs/5V/5VKH/seq_sasa.p')

## distance

In [41]:
# calculate distance
dist_file = p.build_dist()
print(dist_file)

./outputs/5V/5VKH/dist.p


In [42]:
from analyze_dist import AnalyzeDist

max_span = 30
p3= AnalyzeDist(dist_file, max_span=max_span)
p3.print_meta()

number of complex 2
heavy chain A ~ receptor chain C
light chain B ~ receptor chain C


In [43]:
# complex
p3.print_min_dist(0)

A-C
------------------------------
     res_no  res atom      value aa
0         1  GLN    C  21.240194  Q
1         2  VAL    C  20.276529  V
2         3  GLN    C  15.469471  Q
3         4  LEU    C  13.368159  L
4         5  GLN    C  18.786365  Q
..      ...  ...  ...        ... ..
214     215  ILE    C  47.224178  I
215     216  VAL    C  52.014456  V
216     217  PRO    C  55.228643  P
217     218  ARG    C  56.257081  R
218     219  ASP    C  60.267640  D

[219 rows x 5 columns]
------------------------------
    res_no  res atom     value aa
23      45  TYR    C  2.684232  Y
26      48  VAL    C  4.298183  V
27      49  LEU    C  3.578187  L
28      50  ALA    C  4.369733  A
30      52  ARG    C  2.850143  R
31      53  GLY    C  3.726960  G
33      55  PRO    C  3.435817  P
38      60  ILE    C  4.578315  I
39      61  THR    C  3.240289  T
40      62  TYR    C  2.969402  Y
41      63  PRO    C  4.236321  P


In [44]:
# retrieve sequences
motifs = p3.retrieve_seq()
motifs[0]

Sequence-Distance data: ./outputs/5V/5VKH/seq_dist.p.


[{'A': [{'seq': 'WIHWVKQRPGHGLEWIGEIIPS',
    'sig_res': 2,
    'start': (32, 33),
    'end': (53, 54),
    'cutoff': 3},
   {'seq': 'E',
    'sig_res': 1,
    'start': (98, 99),
    'end': (98, 99),
    'cutoff': 3}],
  'C': [{'seq': 'YLAVLAERGAPGAQLITY',
    'sig_res': 3,
    'start': (23, 45),
    'end': (40, 62),
    'cutoff': 3}]},
 {'A': [{'seq': 'TSDWIHWVKQRPGHGLEWIGEIIPSYGRANYNE',
    'sig_res': 10,
    'start': (29, 30),
    'end': (61, 62),
    'cutoff': 4},
   {'seq': 'ERGDG',
    'sig_res': 4,
    'start': (98, 99),
    'end': (102, 103),
    'cutoff': 4}],
  'C': [{'seq': 'YLAVLAERGAPGAQLITY',
    'sig_res': 7,
    'start': (23, 45),
    'end': (40, 62),
    'cutoff': 4}]},
 {'A': [{'seq': 'TSDWIHWVKQRPGHGLEWIGEIIPSYGRANYNE',
    'sig_res': 11,
    'start': (29, 30),
    'end': (61, 62),
    'cutoff': 5},
   {'seq': 'ERGDG',
    'sig_res': 5,
    'start': (98, 99),
    'end': (102, 103),
    'cutoff': 5}],
  'C': [{'seq': 'YLAVLAERGAPGAQLITYP',
    'sig_res': 11,
    'star